# Lazy evaluation on Dask arrays

If you are unfamiliar with Dask, read [Parallel computing with Dask](http://xarray.pydata.org/en/stable/dask.html) in Xarray documentation first. The current version only supports dask arrays on a single machine. Support of [Dask.distributed](https://distributed.dask.org) is in roadmap.

xESMF's Dask support is mostly for [lazy evaluation](https://en.wikipedia.org/wiki/Lazy_evaluation) and [out-of-core computing](https://en.wikipedia.org/wiki/External_memory_algorithm), to allow processing large volumes of data with limited memory. You might also get moderate speed-up on a multi-core machine by [choosing proper chunk sizes](http://xarray.pydata.org/en/stable/dask.html#chunking-and-performance), but that generally won't help your entire pipeline too much, because the read-regrid-write pipeline is severely I/O limited (see [this issue](https://github.com/pangeo-data/pangeo/issues/334) for more discussions). On a single machine, the disk bandwidth is typically limited to ~500 MB/s, and you cannot process data faster than such rate. If you need much faster data processing rate, you should resort to parallel file systems on HPC clusters or distributed storage on public cloud platforms. Please refer to the [Pangeo project](http://pangeo.io/) for more information.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import dask.array as da  # need to have dask.array installed, although not directly using it here.
import xarray as xr
import xesmf as xe

## A simple example

### Prepare input data

In [2]:
ds = xr.tutorial.open_dataset('air_temperature', chunks={'time': 500})
ds

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 53, time: 2920)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(500, 25, 53), meta=np.ndarray>
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [3]:
ds.chunks

Frozen(SortedKeysDict({'time': (500, 500, 500, 500, 500, 420), 'lat': (25,), 'lon': (53,)}))

In [4]:
ds['air'].data

dask.array<open_dataset-6ea21183781926acde7ac39e7ad66665air, shape=(2920, 25, 53), dtype=float32, chunksize=(500, 25, 53), chunktype=numpy.ndarray>

### Build regridder

In [5]:
ds_out = xr.Dataset({'lat': (['lat'], np.arange(16, 75, 1.0)),
                     'lon': (['lon'], np.arange(200, 330, 1.5)),
                    }
                   )

regridder = xe.Regridder(ds, ds_out, 'bilinear')
regridder

xESMF Regridder 
Regridding algorithm:       bilinear 
Input grid shape:           (25, 53) 
Output grid shape:          (59, 87) 
Output grid dimension name: ('lat', 'lon') 
Periodic in longitude?      False

### Apply to xarray Dataset/DataArray

In [6]:
# only build the dask graph; actual computation happens later when calling compute()
%time ds_out = regridder(ds)
ds_out

using dimensions ('lat', 'lon') from data variable air as the horizontal dimensions for this dataset.
CPU times: user 2.09 ms, sys: 0 ns, total: 2.09 ms
Wall time: 2.06 ms


<xarray.Dataset>
Dimensions:  (lat: 59, lon: 87, time: 2920)
Coordinates:
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
  * lon      (lon) float64 200.0 201.5 203.0 204.5 ... 324.5 326.0 327.5 329.0
  * lat      (lat) float64 16.0 17.0 18.0 19.0 20.0 ... 70.0 71.0 72.0 73.0 74.0
Data variables:
    air      (time, lat, lon) float64 dask.array<chunksize=(500, 59, 87), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [7]:
ds_out['air'].data  # chunks are preserved

dask.array<regrid_numpy, shape=(2920, 59, 87), dtype=float64, chunksize=(500, 59, 87), chunktype=numpy.ndarray>

In [8]:
%time result = ds_out['air'].compute()  # actually applies regridding

CPU times: user 298 ms, sys: 212 ms, total: 510 ms
Wall time: 154 ms


In [9]:
type(result.data), result.data.shape

(numpy.ndarray, (2920, 59, 87))

## Invalid chunk sizes to avoid

Dask support relies on `xarray.apply_ufunc`, which requires only chunking over extra/broadcasting dimensions (like `time` and `lev`), not horizontal/core dimensions (`lat`, `lon`, or `x`, `y`).

In [10]:
# xESMF doesn't like chunking over horizontal dimensions
ds_bad = ds.chunk({'lat': 10, 'lon': 10, 'time': None})
ds_bad

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 53, time: 2920)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(2920, 10, 10), meta=np.ndarray>
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [11]:
# regridder(ds_bad)  # uncomment this line to see the error message

In [12]:
# besides rechunking data properly, another simple fix is to convert to numpy array without chunking
regridder(ds_bad.load())

using dimensions ('lat', 'lon') from data variable air as the horizontal dimensions for this dataset.


<xarray.Dataset>
Dimensions:  (lat: 59, lon: 87, time: 2920)
Coordinates:
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
  * lon      (lon) float64 200.0 201.5 203.0 204.5 ... 324.5 326.0 327.5 329.0
  * lat      (lat) float64 16.0 17.0 18.0 19.0 20.0 ... 70.0 71.0 72.0 73.0 74.0
Data variables:
    air      (time, lat, lon) float64 296.1 296.4 296.6 ... 240.9 241.0 241.5
Attributes:
    regrid_method:  bilinear